# Fabric-Vigil Deployment

Deploys the Vigil mirror monitoring solution into the current workspace. Run this notebook
once per workspace that contains mirrored databases you want to monitor.

## What Gets Deployed

| Item | Name | Purpose |
|------|------|---------|
| Notebook | `nb_vigil_mirror_monitoring` | Checks all mirrored database table health |
| Pipeline | `pl_vigil_mirror_monitoring` | Runs the notebook on a schedule |
| Schedule | (Cron) | Triggers the pipeline at a configurable interval |
| Reflex | `da_vigil_mirror_monitoring` | Sends email alert when the pipeline fails |

## Prerequisites

### 1. Workspace Permissions

The user running this notebook must have **Contributor** or **Admin** role on the target workspace.

### 2. Fabric Capacity

The target workspace must be on a Fabric capacity (F-SKU or Trial).
Data Activator (Reflex) items require Fabric capacity.

### 3. (Optional) Service Principal Setup

These steps are only required if you switch `AUTH_MODE` to `"spn"` in the
configuration cell below.

**Entra ID App Registration:**
1. Navigate to **Entra ID > App registrations > New registration**
2. Name it (e.g., `spn-fabric-vigil`) and register
3. Under **Certificates & secrets**, create a new client secret and save the value
4. Note the **Application (client) ID** and **Directory (tenant) ID**

**Fabric Tenant Settings** (requires a Fabric Administrator):

| Setting | Location |
|---------|----------|
| Service principals can use Fabric APIs | Developer settings |
| Allow service principals to create and use profiles | Developer settings |

For each setting, add a security group that contains your SPN.

**Workspace Access:** The SPN must have **Contributor** (or higher) role on the target workspace.

**Azure Key Vault (Recommended for SPN):**
Store the SPN credentials as Key Vault secrets:
- `fabric-spn-client-id`: The Application (client) ID
- `fabric-spn-client-secret`: The client secret value

The notebook runtime identity must have **Key Vault Secrets User** role or a
Key Vault access policy granting **Get** on secrets.


## Imports

In [ ]:
import json
import base64
import requests
import time
from datetime import datetime, timezone

## Configuration

Set deployment parameters below. At minimum, configure `KEY_VAULT_NAME` (recommended)
or the hardcoded credentials for SPN authentication.


In [ ]:
# ==============================================================================
# AUTHENTICATION
# ==============================================================================

# Authentication mode: "user" (default) or "spn"
#   "user" - Items are owned by your personal identity (simplest setup)
#   "spn"  - Items are owned by a service principal (requires Entra app registration)
AUTH_MODE = "user"

# ==============================================================================
# KEY VAULT CONFIGURATION (recommended)
# ==============================================================================
# Set KEY_VAULT_NAME to retrieve SPN credentials from Azure Key Vault.
# Expected secrets: "fabric-spn-client-id", "fabric-spn-client-secret"
KEY_VAULT_NAME = ""  # e.g., "kv-fabric-prod"

# ==============================================================================
# HARDCODED CREDENTIALS (development/testing ONLY)
# WARNING: Never commit secrets to source control.
# These are only used if Key Vault is not configured.
# ==============================================================================
SP_CLIENT_ID = ""
SP_CLIENT_SECRET = ""

# ==============================================================================
# SCHEDULE
# ==============================================================================

# How often (in minutes) the monitoring pipeline runs
SCHEDULE_INTERVAL_MINUTES = 30

# Timezone for the schedule
SCHEDULE_TIMEZONE = "Central Standard Time"

# ==============================================================================
# ALERT RECIPIENT
# ==============================================================================

# Leave empty to auto-detect the executing user's email address.
# Set explicitly if alerts should go to someone else (e.g., a team DL).
ALERT_EMAIL_OVERRIDE = ""

# ==============================================================================
# GITHUB SOURCE (do not change unless you have forked the repo)
# ==============================================================================
GITHUB_REPO = "imtkain/Fabric-Vigil"
GITHUB_BRANCH = "main"
GITHUB_FILES_PATH = "files"

# ==============================================================================
# ITEM DISPLAY NAMES (customize if desired)
# ==============================================================================
NOTEBOOK_NAME = "nb_vigil_mirror_monitoring"
PIPELINE_NAME = "pl_vigil_mirror_monitoring"
REFLEX_NAME = "da_vigil_mirror_monitoring"


## Core Functions

Logging, JWT decoding, authentication, and API helpers.

In [ ]:
def log(message: str) -> None:
    """Print a timestamped status message."""
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {message}")


def decode_jwt_claims(token: str) -> dict:
    """
    Decode the payload claims from a JWT access token.

    Does not verify the signature (unnecessary for extracting claims
    from a token we just acquired from a trusted authority).
    """
    payload_b64 = token.split(".")[1]
    # Add padding for base64 decoding
    payload_b64 += "=" * (4 - len(payload_b64) % 4)
    return json.loads(base64.b64decode(payload_b64))


def get_user_context() -> dict:
    """
    Extract the executing user's UPN and tenant ID from a user-delegated token.

    Always uses the interactive user's identity (via notebookutils), regardless
    of AUTH_MODE. This captures the human who initiated the deployment for use
    as the default alert recipient.

    Returns:
        dict: {"upn": str, "tenant_id": str}
    """
    log("Detecting user context from notebook identity...")
    raw_token = notebookutils.credentials.getToken("https://api.fabric.microsoft.com")
    token = raw_token.token if hasattr(raw_token, "token") else raw_token
    claims = decode_jwt_claims(token)

    upn = claims.get("upn") or claims.get("preferred_username")
    tenant_id = claims.get("tid")

    if not upn:
        raise ValueError(
            "Could not detect UPN from token claims. "
            "Set ALERT_EMAIL_OVERRIDE in the configuration cell."
        )
    if not tenant_id:
        raise ValueError("Could not detect tenant ID (tid) from token claims.")

    log(f"  UPN:       {upn}")
    log(f"  Tenant ID: {tenant_id}")
    return {"upn": upn, "tenant_id": tenant_id}


def get_auth_headers(tenant_id: str) -> dict:
    """
    Acquire an access token based on AUTH_MODE and return HTTP headers.

    SPN mode: Uses MSAL client credentials flow. Credentials are resolved
    from Key Vault first, then hardcoded values as fallback.

    User mode: Uses notebookutils to get a token for the interactive user.

    Args:
        tenant_id: Entra tenant ID (detected from user context).

    Returns:
        dict: Headers with Authorization bearer token and Content-Type.
    """
    if AUTH_MODE == "spn":
        log("Authenticating as service principal...")

        # Resolve credentials: Key Vault takes priority over hardcoded values
        client_id = SP_CLIENT_ID
        client_secret = SP_CLIENT_SECRET

        if KEY_VAULT_NAME:
            kv_url = f"https://{KEY_VAULT_NAME}.vault.azure.net/"
            log(f"  Retrieving credentials from Key Vault ({KEY_VAULT_NAME})...")
            if not client_id:
                client_id = notebookutils.credentials.getSecret(kv_url, "fabric-spn-client-id")
            if not client_secret:
                client_secret = notebookutils.credentials.getSecret(kv_url, "fabric-spn-client-secret")

        if not client_id or not client_secret:
            raise ValueError(
                "SPN authentication requires client ID and client secret. "
                "Configure KEY_VAULT_NAME or set SP_CLIENT_ID and SP_CLIENT_SECRET."
            )

        from msal import ConfidentialClientApplication
        app = ConfidentialClientApplication(
            client_id=client_id,
            client_credential=client_secret,
            authority=f"https://login.microsoftonline.com/{tenant_id}"
        )

        result = app.acquire_token_for_client(
            scopes=["https://api.fabric.microsoft.com/.default"]
        )

        if "access_token" not in result:
            error_desc = result.get("error_description", "Unknown error")
            raise Exception(f"SPN token acquisition failed: {error_desc}")

        token = result["access_token"]
        log("  SPN token acquired")

    elif AUTH_MODE == "user":
        log("Authenticating as current user...")
        raw_token = notebookutils.credentials.getToken("https://api.fabric.microsoft.com")
        token = raw_token.token if hasattr(raw_token, "token") else raw_token
        log("  User token acquired")

    else:
        raise ValueError(f"Invalid AUTH_MODE: '{AUTH_MODE}'. Must be 'spn' or 'user'.")

    return {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }


def wait_for_lro(
    headers: dict,
    response: requests.Response,
    item_desc: str,
    timeout_seconds: int = 300
) -> dict:
    """
    Poll a Fabric long-running operation (LRO) until completion.

    Fabric returns 202 Accepted with a Location header for async item creation.
    This function polls that URL until the operation succeeds, fails, or times out.

    Args:
        headers: Auth headers for polling requests.
        response: The initial 202 response.
        item_desc: Description for log messages (e.g., "notebook").
        timeout_seconds: Max wait time before raising an error.

    Returns:
        dict: The final operation result JSON.
    """
    location = response.headers.get("Location")
    retry_after = int(response.headers.get("Retry-After", 5))

    if not location:
        raise Exception(f"202 response for {item_desc} but no Location header")

    log(f"  Waiting for {item_desc} creation (polling every {retry_after}s)...")

    start = time.time()
    while time.time() - start < timeout_seconds:
        time.sleep(retry_after)
        poll_resp = requests.get(location, headers=headers)

        if poll_resp.status_code == 200:
            result = poll_resp.json()
            status = result.get("status", "").lower()

            if status in ("succeeded", "completed"):
                log(f"  {item_desc} creation completed")
                return result
            elif status in ("failed", "cancelled"):
                error = result.get("error", {}).get("message", "Unknown error")
                raise Exception(f"{item_desc} creation failed: {error}")

        elapsed = int(time.time() - start)
        log(f"    ... in progress ({elapsed}s elapsed)")

    raise Exception(f"{item_desc} creation timed out after {timeout_seconds}s")


def find_item_id(
    headers: dict,
    workspace_id: str,
    item_name: str,
    item_type: str
) -> str:
    """
    Look up an item's ID by display name and type in the workspace.

    Used after LRO completion when the create response does not include
    the item ID directly.

    Args:
        headers: Auth headers.
        workspace_id: Workspace to search.
        item_name: Display name to match.
        item_type: Fabric item type (e.g., "Notebook", "DataPipeline", "Reflex").

    Returns:
        str: The item ID.
    """
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    for item in resp.json().get("value", []):
        if item.get("displayName") == item_name and item.get("type") == item_type:
            item_id = item.get("id")
            log(f"  Found {item_type} '{item_name}' (ID: {item_id})")
            return item_id

    raise Exception(f"Could not find {item_type} '{item_name}' in workspace after creation")


def download_github_file(file_path: str) -> bytes:
    """
    Download a file from the Fabric-Vigil GitHub repository.

    Uses raw.githubusercontent.com for direct file access.

    Args:
        file_path: Path relative to repo root (e.g., "files/pipeline_content.json").

    Returns:
        bytes: Raw file content.
    """
    url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{GITHUB_BRANCH}/{file_path}"
    log(f"  Downloading {file_path}...")
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.content


def create_fabric_item(
    headers: dict,
    url: str,
    body: dict,
    item_desc: str,
    workspace_id: str,
    item_name: str,
    item_type: str
) -> str:
    """
    Create a Fabric item via REST API, handling both sync and async responses.

    Args:
        headers: Auth headers.
        url: The creation endpoint URL.
        body: The JSON request body.
        item_desc: Human-readable description for logging.
        workspace_id: Workspace ID (for item lookup after LRO).
        item_name: Expected display name (for item lookup after LRO).
        item_type: Expected Fabric type (for item lookup after LRO).

    Returns:
        str: The created item's ID.
    """
    resp = requests.post(url, headers=headers, json=body)

    if resp.status_code in (200, 201):
        item_id = resp.json().get("id")
        log(f"  {item_desc} created (ID: {item_id})")
        return item_id
    elif resp.status_code == 202:
        wait_for_lro(headers, resp, item_desc)
        return find_item_id(headers, workspace_id, item_name, item_type)
    else:
        # Surface the error details before raising
        try:
            error_body = resp.json()
            log(f"  Error: {json.dumps(error_body, indent=2)}")
        except Exception:
            log(f"  Error: {resp.status_code} {resp.text}")
        resp.raise_for_status()


## Deployment Functions

Each function downloads a template from GitHub, replaces placeholders, and creates the item.

In [ ]:
def deploy_notebook(headers: dict, workspace_id: str) -> str:
    """
    Download the Vigil monitoring notebook from GitHub and deploy it.

    The notebook has no placeholder GUIDs; it discovers everything
    dynamically at runtime.

    Returns:
        str: The created notebook's item ID.
    """
    log("=" * 60)
    log("DEPLOYING NOTEBOOK")
    log("=" * 60)

    nb_content = download_github_file(f"{GITHUB_FILES_PATH}/nb_vigil_mirror_monitoring.ipynb")
    nb_b64 = base64.b64encode(nb_content).decode("utf-8")

    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/notebooks"
    body = {
        "displayName": NOTEBOOK_NAME,
        "definition": {
            "format": "ipynb",
            "parts": [
                {
                    "path": "notebook-content.ipynb",
                    "payload": nb_b64,
                    "payloadType": "InlineBase64"
                }
            ]
        }
    }

    return create_fabric_item(
        headers, url, body, "Notebook",
        workspace_id, NOTEBOOK_NAME, "Notebook"
    )


def deploy_pipeline(headers: dict, workspace_id: str, notebook_id: str) -> str:
    """
    Download the pipeline template, inject the notebook GUID, and deploy.

    Replacements:
        __NOTEBOOK_GUID__ -> notebook_id
        __WORKSPACE_GUID__ -> workspace_id

    Returns:
        str: The created pipeline's item ID.
    """
    log("=" * 60)
    log("DEPLOYING PIPELINE")
    log("=" * 60)

    # Download and apply replacements to pipeline content
    content_raw = download_github_file(f"{GITHUB_FILES_PATH}/pipeline_content.json")
    content_str = content_raw.decode("utf-8")
    content_str = content_str.replace("__NOTEBOOK_GUID__", notebook_id)
    content_str = content_str.replace("__WORKSPACE_GUID__", workspace_id)
    content_b64 = base64.b64encode(content_str.encode("utf-8")).decode("utf-8")

    # Download platform metadata (no replacements needed)
    platform_raw = download_github_file(f"{GITHUB_FILES_PATH}/pipeline_platform.json")
    platform_b64 = base64.b64encode(platform_raw).decode("utf-8")

    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/dataPipelines"
    body = {
        "displayName": PIPELINE_NAME,
        "definition": {
            "parts": [
                {
                    "path": "pipeline-content.json",
                    "payload": content_b64,
                    "payloadType": "InlineBase64"
                },
                {
                    "path": ".platform",
                    "payload": platform_b64,
                    "payloadType": "InlineBase64"
                }
            ]
        }
    }

    return create_fabric_item(
        headers, url, body, "Pipeline",
        workspace_id, PIPELINE_NAME, "DataPipeline"
    )


def create_pipeline_schedule(headers: dict, workspace_id: str, pipeline_id: str) -> str:
    """
    Create a Cron schedule on the deployed pipeline.

    Uses the Job Scheduler API. The schedule starts immediately and runs
    at the interval specified by SCHEDULE_INTERVAL_MINUTES.

    Returns:
        str: The schedule ID.
    """
    log("=" * 60)
    log(f"CREATING SCHEDULE (every {SCHEDULE_INTERVAL_MINUTES} min)")
    log("=" * 60)

    start_dt = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S")
    end_dt = "2036-12-31T23:59:59"

    url = (
        f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}"
        f"/items/{pipeline_id}/jobs/Pipeline/schedules"
    )
    body = {
        "enabled": True,
        "configuration": {
            "startDateTime": start_dt,
            "endDateTime": end_dt,
            "localTimeZoneId": SCHEDULE_TIMEZONE,
            "type": "Cron",
            "interval": SCHEDULE_INTERVAL_MINUTES
        }
    }

    resp = requests.post(url, headers=headers, json=body)

    if resp.status_code in (200, 201):
        schedule_id = resp.json().get("id")
        log(f"  Schedule created (ID: {schedule_id})")
        return schedule_id
    else:
        try:
            error_body = resp.json()
            log(f"  Error: {json.dumps(error_body, indent=2)}")
        except Exception:
            log(f"  Error: {resp.status_code} {resp.text}")
        resp.raise_for_status()


def deploy_reflex(
    headers: dict,
    workspace_id: str,
    pipeline_id: str,
    tenant_id: str,
    alert_email: str
) -> str:
    """
    Download the reflex template, inject pipeline GUID, tenant, email, and deploy.

    Replacements:
        __PIPELINE_GUID__ -> pipeline_id
        __WORKSPACE_GUID__ -> workspace_id
        __TENANT_GUID__    -> tenant_id
        __USER_UPN__       -> alert_email

    Returns:
        str: The created reflex's item ID.
    """
    log("=" * 60)
    log("DEPLOYING DATA ACTIVATOR (REFLEX)")
    log("=" * 60)

    # Download and apply replacements to reflex entities
    entities_raw = download_github_file(f"{GITHUB_FILES_PATH}/reflex_entities.json")
    entities_str = entities_raw.decode("utf-8")
    entities_str = entities_str.replace("__PIPELINE_GUID__", pipeline_id)
    entities_str = entities_str.replace("__WORKSPACE_GUID__", workspace_id)
    entities_str = entities_str.replace("__TENANT_GUID__", tenant_id)
    entities_str = entities_str.replace("__USER_UPN__", alert_email)
    entities_b64 = base64.b64encode(entities_str.encode("utf-8")).decode("utf-8")

    # Download platform metadata (no replacements needed)
    platform_raw = download_github_file(f"{GITHUB_FILES_PATH}/reflex_platform.json")
    platform_b64 = base64.b64encode(platform_raw).decode("utf-8")

    url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/reflexes"
    body = {
        "displayName": REFLEX_NAME,
        "definition": {
            "parts": [
                {
                    "path": "ReflexEntities.json",
                    "payload": entities_b64,
                    "payloadType": "InlineBase64"
                },
                {
                    "path": ".platform",
                    "payload": platform_b64,
                    "payloadType": "InlineBase64"
                }
            ]
        }
    }

    return create_fabric_item(
        headers, url, body, "Reflex",
        workspace_id, REFLEX_NAME, "Reflex"
    )


## Deploy

Run the cell below to deploy the full Vigil solution into this workspace.

In [ ]:
# ==============================================================================
# MAIN DEPLOYMENT
# ==============================================================================
deploy_start = time.time()

log("=" * 60)
log("FABRIC-VIGIL DEPLOYMENT")
log("=" * 60)

# Step 1: Detect user context (UPN and tenant ID from the interactive user)
user_context = get_user_context()
tenant_id = user_context["tenant_id"]
alert_email = ALERT_EMAIL_OVERRIDE if ALERT_EMAIL_OVERRIDE else user_context["upn"]
log(f"Alert recipient: {alert_email}")

# Step 2: Get workspace context
runtime_ctx = notebookutils.runtime.context
workspace_id = runtime_ctx.get("currentWorkspaceId") or runtime_ctx.get("workspaceId")
if not workspace_id:
    raise ValueError("Could not determine workspace ID from notebookutils.runtime.context")
log(f"Target workspace: {workspace_id}")

# Step 3: Authenticate for API calls (SPN or user, per AUTH_MODE)
headers = get_auth_headers(tenant_id)

# Step 4: Deploy items in dependency order
notebook_id = deploy_notebook(headers, workspace_id)
pipeline_id = deploy_pipeline(headers, workspace_id, notebook_id)
schedule_id = create_pipeline_schedule(headers, workspace_id, pipeline_id)
reflex_id = deploy_reflex(headers, workspace_id, pipeline_id, tenant_id, alert_email)

# Summary
elapsed = time.time() - deploy_start
log("")
log("=" * 60)
log("DEPLOYMENT COMPLETE")
log("=" * 60)
log(f"  Notebook:  {NOTEBOOK_NAME} ({notebook_id})")
log(f"  Pipeline:  {PIPELINE_NAME} ({pipeline_id})")
log(f"  Schedule:  Every {SCHEDULE_INTERVAL_MINUTES} min, {SCHEDULE_TIMEZONE} ({schedule_id})")
log(f"  Reflex:    {REFLEX_NAME} ({reflex_id})")
log(f"  Alerts to: {alert_email}")
log(f"  Elapsed:   {elapsed:.1f}s")
log("=" * 60)
